## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head(5)

,City_ID,city,country,latitude,longitude,Maximum_temperature,percent_humidity,percent_cloudiness,wind_speed,description
0,0,Agüimes,ES,27.9054,-15.4461,20.35,78,75,9.77,broken clouds
1,1,Ancud,CL,-41.8697,-73.8203,6.74,92,8,1.92,clear sky
2,2,Cayenne,GF,4.9333,-52.3333,24.49,90,0,0.00,clear sky
3,3,Damāvand,IR,35.7178,52.0650,19.91,62,45,2.41,scattered clouds
4,4,Isabela,PH,6.7085,121.9711,31.03,59,84,1.92,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 23
What is the maximum temperature you would like for your trip? 32


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df["Maximum_temperature"] <= max_temp) & 
                                       (city_data_df["Maximum_temperature"] >= min_temp)]
preferred_cities_df.head()

,City_ID,city,country,latitude,longitude,Maximum_temperature,percent_humidity,percent_cloudiness,wind_speed,description
2,2,Cayenne,GF,4.9333,-52.3333,24.49,90,0,0.00,clear sky
4,4,Isabela,PH,6.7085,121.9711,31.03,59,84,1.92,broken clouds
8,8,Urucurituba,BR,-2.6833,-57.6667,26.61,56,69,0.36,broken clouds
12,12,Arlit,NE,18.7369,7.3853,30.08,46,86,4.13,overcast clouds
16,16,Venice,US,27.0998,-82.4543,29.01,87,40,2.06,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                240
city                   240
country                240
latitude               240
longitude              240
Maximum_temperature    240
percent_humidity       240
percent_cloudiness     240
wind_speed             240
description            240
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df
clean_df

,City_ID,city,country,latitude,longitude,Maximum_temperature,percent_humidity,percent_cloudiness,wind_speed,description
2,2,Cayenne,GF,4.9333,-52.3333,24.49,90,0,0.00,clear sky
4,4,Isabela,PH,6.7085,121.9711,31.03,59,84,1.92,broken clouds
8,8,Urucurituba,BR,-2.6833,-57.6667,26.61,56,69,0.36,broken clouds
12,12,Arlit,NE,18.7369,7.3853,30.08,46,86,4.13,overcast clouds
16,16,Venice,US,27.0998,-82.4543,29.01,87,40,2.06,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
713,713,Bajil,YE,15.0584,43.2851,30.90,58,69,1.90,broken clouds
717,717,Novo Aripuanã,BR,-5.1206,-60.3797,23.47,53,5,1.29,clear sky
721,721,Waddān,LY,29.1614,16.1390,23.19,60,0,1.63,clear sky
724,724,Dāmghān,IR,36.1683,54.3480,25.20,51,0,4.82,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["city", "country", "Maximum_temperature", "description", "latitude", "longitude"]].copy()


# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,city,country,Maximum_temperature,description,latitude,longitude,Hotel Name
2,Cayenne,GF,24.49,clear sky,4.9333,-52.3333,
4,Isabela,PH,31.03,broken clouds,6.7085,121.9711,
8,Urucurituba,BR,26.61,broken clouds,-2.6833,-57.6667,
12,Arlit,NE,30.08,overcast clouds,18.7369,7.3853,
16,Venice,US,29.01,scattered clouds,27.0998,-82.4543,
17,Port-Gentil,GA,24.44,broken clouds,-0.7193,8.7815,
19,Mitha Tiwana,PK,31.16,scattered clouds,32.2486,72.1108,
25,St. Marys,US,26.01,clear sky,30.7305,-81.5465,
32,Kavaratti,IN,27.92,overcast clouds,10.5669,72.6420,
34,Shāhābād,IN,30.98,overcast clouds,27.6500,79.9500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}



for index, row in hotel_df.iterrows():
    

    # Get the latitude and longitude.
    lat = row["latitude"]
    lng = row["longitude"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("Hotel found for location" + str(index))
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel found for location2
Hotel found for location4
Hotel not found... skipping.
Hotel found for location12
Hotel found for location16
Hotel found for location17
Hotel found for location19
Hotel found for location25
Hotel found for location32
Hotel found for location34
Hotel found for location36
Hotel found for location46
Hotel found for location49
Hotel found for location55
Hotel found for location57
Hotel found for location68
Hotel found for location69
Hotel found for location70
Hotel found for location73
Hotel found for location77
Hotel found for location78
Hotel found for location84
Hotel found for location85
Hotel found for location87
Hotel not found... skipping.
Hotel found for location92
Hotel found for location93
Hotel found for location97
Hotel found for location101
Hotel not found... skipping.
Hotel found for location110
Hotel found for location111
Hotel found for location113
Hotel found for location119
Hotel found for location122
Hotel found for location124
Hotel found for l

In [9]:
hotel_df.head(10)



,city,country,Maximum_temperature,description,latitude,longitude,Hotel Name
2,Cayenne,GF,24.49,clear sky,4.9333,-52.3333,Hôtel Le Dronmi
4,Isabela,PH,31.03,broken clouds,6.7085,121.9711,Isawad Resort
8,Urucurituba,BR,26.61,broken clouds,-2.6833,-57.6667,
12,Arlit,NE,30.08,overcast clouds,18.7369,7.3853,Case de Passage De la Prefecture
16,Venice,US,29.01,scattered clouds,27.0998,-82.4543,A Beach Retreat On Casey Key
17,Port-Gentil,GA,24.44,broken clouds,-0.7193,8.7815,Fengshui Residence
19,Mitha Tiwana,PK,31.16,scattered clouds,32.2486,72.1108,Canal Rest House
25,St. Marys,US,26.01,clear sky,30.7305,-81.5465,Spencer House Inn
32,Kavaratti,IN,27.92,overcast clouds,10.5669,72.6420,Hotels in Lakshadweep Islands
34,Shāhābād,IN,30.98,overcast clouds,27.6500,79.9500,डाक बंगला


In [10]:
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df

,city,country,Maximum_temperature,description,latitude,longitude,Hotel Name
2,Cayenne,GF,24.49,clear sky,4.9333,-52.3333,Hôtel Le Dronmi
4,Isabela,PH,31.03,broken clouds,6.7085,121.9711,Isawad Resort
8,Urucurituba,BR,26.61,broken clouds,-2.6833,-57.6667,NaN
12,Arlit,NE,30.08,overcast clouds,18.7369,7.3853,Case de Passage De la Prefecture
16,Venice,US,29.01,scattered clouds,27.0998,-82.4543,A Beach Retreat On Casey Key
...,...,...,...,...,...,...,...
713,Bajil,YE,30.90,broken clouds,15.0584,43.2851,فندق حضرموت
717,Novo Aripuanã,BR,23.47,clear sky,-5.1206,-60.3797,Hotel Ezequiel
721,Waddān,LY,23.19,clear sky,29.1614,16.1390,محل الشريف محمد العايب
724,Dāmghān,IR,25.20,clear sky,36.1683,54.3480,خوابگاه اساتيد دانشگاه دامغان


In [11]:
hotel_df.dropna()

,city,country,Maximum_temperature,description,latitude,longitude,Hotel Name
2,Cayenne,GF,24.49,clear sky,4.9333,-52.3333,Hôtel Le Dronmi
4,Isabela,PH,31.03,broken clouds,6.7085,121.9711,Isawad Resort
12,Arlit,NE,30.08,overcast clouds,18.7369,7.3853,Case de Passage De la Prefecture
16,Venice,US,29.01,scattered clouds,27.0998,-82.4543,A Beach Retreat On Casey Key
17,Port-Gentil,GA,24.44,broken clouds,-0.7193,8.7815,Fengshui Residence
...,...,...,...,...,...,...,...
713,Bajil,YE,30.90,broken clouds,15.0584,43.2851,فندق حضرموت
717,Novo Aripuanã,BR,23.47,clear sky,-5.1206,-60.3797,Hotel Ezequiel
721,Waddān,LY,23.19,clear sky,29.1614,16.1390,محل الشريف محمد العايب
724,Dāmghān,IR,25.20,clear sky,36.1683,54.3480,خوابگاه اساتيد دانشگاه دامغان


In [12]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>

<dt>city</dt><dd>{city}</dd>
<dt>country</dt><dd>{country}</dd>
<dt>Maximum_temperature</dt><dd>{Maximum_temperature} °F</dd>
<dt>description</dt><dd>{description} </dd>

</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_df[["latitude", "longitude"]]
max_temp = hotel_df["Maximum_temperature"]


In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>

<dt>city</dt><dd>{city}</dd>
<dt>country</dt><dd>{country}</dd>
<dt>Maximum_temperature</dt><dd>{Maximum_temperature} °F</dd>
<dt>description</dt><dd>{description} </dd>

</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_df[["latitude", "longitude"]]


done


In [20]:
# 11a. Add a marker layer for each city to the map. 

max_temp = hotel_df["Maximum_temperature"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))